In [1]:
from app_function import *

## KNN Basic Model

In [2]:
# Loading data
df = pd.read_csv("data/full_data_cleaned.csv")

#correct type in data
df["eye_color"] = df["eye_color"].replace("gray", "grey")
df["eye_color"] = df["eye_color"].replace("Grey", "grey")
df["hair_color"] = df["hair_color"].replace("gray", "grey")
df["skin_tone"] = df["skin_tone"].replace("notSureST", "Not Sure")

#Product data
products = pd.read_csv("data/products.csv")

# Filter the DataFrame to retain only users with at least two predictions
filtered_df = filter_users_with_enough_predictions(df[['author_id', 'product_id', 'rating_x']], min_predictions=2)
# Building Surprise train dataset
reader = Reader(rating_scale=(1, 5))
load_data = Dataset.load_from_df(filtered_df[['author_id', 'product_id', 'rating_x']], reader)
# Store best parameters
best_parameters =  {
    'k': 50,
    'sim_options': {
        'name': 'pearson',
        'min_support': 5,
        'user_based': False,
        'shrinkage': 0
    },
}
KNNBasicrecommender = KNNBasic(k= best_parameters["k"], 
                               sim_options = best_parameters["sim_options"])
#Train the model
train_set = load_data.build_full_trainset()
anti_set = load_data.build_full_trainset().build_anti_testset()
KNNBasicrecommender.fit(train_set)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [3]:
# Check antiset
anti_set

[('1238130325', 'P7880', 4.37528402394758),
 ('1238130325', 'P218700', 4.37528402394758),
 ('1238130325', 'P248407', 4.37528402394758),
 ('1238130325', 'P269122', 4.37528402394758),
 ('1238130325', 'P417238', 4.37528402394758),
 ('1238130325', 'P450271', 4.37528402394758),
 ('1238130325', 'P427421', 4.37528402394758),
 ('1238130325', 'P411387', 4.37528402394758),
 ('1238130325', 'P441644', 4.37528402394758),
 ('1238130325', 'P427417', 4.37528402394758),
 ('1238130325', 'P309308', 4.37528402394758),
 ('1238130325', 'P411540', 4.37528402394758),
 ('1238130325', 'P471546', 4.37528402394758),
 ('1238130325', 'P248404', 4.37528402394758),
 ('1238130325', 'P443352', 4.37528402394758),
 ('1238130325', 'P423688', 4.37528402394758),
 ('1238130325', 'P421275', 4.37528402394758),
 ('1238130325', 'P297524', 4.37528402394758),
 ('1238130325', 'P429952', 4.37528402394758),
 ('1238130325', 'P480630', 4.37528402394758),
 ('1238130325', 'P126301', 4.37528402394758),
 ('1238130325', 'P453816', 4.3752840

In [4]:
# Check if our function works
get_top_n_recommendations(KNNBasicrecommender, '1238130325', anti_set, 5, df)

,product_name,product_id,brand,limited_edition,price,rating,niche
0,Dreamskin Skin Perfector,P415619,Dior,0,130.0,4.324653,0
1,Vitamin C Lactic 15% Firm & Bright Serum,P501169,Dr. Dennis Gross Skincare,0,85.0,4.723077,1
2,Skin Smoothing Cream Moisturizer,P443359,Dermalogica,0,74.0,4.718631,0
3,Pimple Paste Overnight Blemish Drying Paste,P471039,iNNBEAUTY PROJECT,0,17.0,4.396947,1
4,Set The Stage Hydrating Primer Serum,P458546,LAWLESS,0,32.0,4.680000,1


## Build Application

In [5]:
#---------------------------------Navigation & Carousel---------------------------------#
#store images
images = [
    "https://img.freepik.com/free-photo/top-view-salts-cream-container_23-2148899419.jpg?t=st=1713040942~exp=1713044542~hmac=3029e170df64d4468c49285420becbe99b6e86a3123bd0e97f9a4e31e985b335&w=1380",
    "https://img.freepik.com/free-photo/flat-lay-natural-cream-concept_23-2148565353.jpg?t=st=1713041064~exp=1713044664~hmac=20f59a3f453de2038a656b2ab081f725d6d0b039f2ab08feaf7d3be6aa59857c&w=1380",
]
# Navigation bar
Navigation = dbc.Navbar(
        dbc.Container([
            html.A(
                dbc.Row(
                    [dbc.NavbarBrand("Forever Young Global", className="ml-auto") 
                                     #style={"fontFamily": "Arial, sans-serif"})
                     ],
                    align="center"
                ),
                href="#",
            ),
            dbc.NavbarToggler(id="navbar-toggler"),
            dbc.Collapse(
                dbc.Nav(
                    [dbc.NavItem(dbc.NavLink("Home", href="#")), 
                     dbc.NavItem(dbc.NavLink("New Arrivals", href="#")), 
                     dbc.NavItem(dbc.NavLink("Skincare", href="#")),
                     dbc.NavItem(dbc.NavLink("Makeup", href="#")),
                     dbc.NavItem(dbc.NavLink("Body", href="#")),
                     dbc.NavItem(dbc.NavLink("Hair", href="#")),],
                    className="ml-auto", navbar=True),
                id="navbar-collapse",
                navbar=True,
            ),
        ]),
        color="light",
        light=True,
        sticky="top"
    )

#Carousel of images
Carousel = dbc.Container(
        dbc.Carousel(
        items=[
            {"key": i, "src": img} for i, img in enumerate(images)
        ],
        controls=True,
        indicators=True,
        interval=3000,
        className= "carousel-fade",
    ),
        style = {
            'width': '100%',
            'maxHeight' : '850px'
        }
)

#---------------------------------Top Trending and Top Rated table---------------#

# Create table for top n trending products
Top_n_trending = get_top_n_trending()
# Create table for top n rated products
Top_n_rated = get_top_n_rated()

#---------------------------------Dropdown menu---------------------------------#
#User id dropdown
##Create options of user ids
options_user_ids = [{'label': str(user), 'value': user} for user in df['author_id'].unique()]

##Dropdown for user ids
user_id_dropdown = dcc.Dropdown(
    id='user_id_dropdown',
    options= options_user_ids,
    value= '',
    style = {'width': '50%'},
    clearable= True,
    multi=False,
)

# Hair color dropdown
##Create options of Hair color
options_hair = [{'label': color, 'value': color} for color in df['hair_color'].unique()]
options_hair = [x for x in options_hair if str(x['label']) != 'nan'] # Remove nan values
##Dropdown for hair color
hair_color_dropdown = dcc.Dropdown(
    id='hair_color_dropdown',
    options= options_hair,
    value= '',
    style = {'width': '50%'},
    clearable= True,
    multi= True,
)

# Skin type dropdown
##Create options of skin types
options_skin = [{'label': skin, 'value': skin} for skin in df['skin_type'].unique()]
options_skin = [x for x in options_skin if str(x['label']) != 'nan'] # Remove nan values
##Dropdown for skin type
skin_type_dropdown = dcc.Dropdown(
    id='skin_type_dropdown',
    options= options_skin,
    value= '',
    style = {'width': '50%'},
    clearable= True,
    multi= True,
)

# Skin tone dropdown
##Create options of skin tones
options_tone = [{'label': tone, 'value': tone} for tone in df['skin_tone'].unique()]
options_tone = [x for x in options_tone if str(x['label']) != 'nan'] # Remove nan values
##Dropdown for skin tone
skin_tone_dropdown = dcc.Dropdown(
    id='skin_tone_dropdown',
    options= options_tone,
    value= '',
    style = {'width': '50%'},
    clearable= True,
    multi= True,
)

# Eye color dropdown
##Create options of eye colors
options_eye = [{'label': eye, 'value': eye} for eye in df['eye_color'].unique()]
options_eye = [x for x in options_eye if str(x['label']) != 'nan'] # Remove nan values
##Dropdown for eye color
eye_color_dropdown = dcc.Dropdown(
    id='eye_color_dropdown',
    options= options_eye,
    value= '',
    style = {'width': '50%'},
    clearable= True,
    multi= True,
)

#Price slider
price_slider = dcc.RangeSlider(id = 'input-price', 
                          min = df['price_usd_x'].min(), max = df['price_usd_x'].max(),
                          value=[df['price_usd_x'].min(), df['price_usd_x'].max()])

# Niche Options
options_niche = [{'label' : 'I love niche products', 'value' : 2},
                 {'label' : "No Thanks", 'value' : 1}]
niche_dropdown = dcc.Dropdown(
    id='niche_dropdown',
    options= options_niche,
    value= '',
    style = {'width': '50%'},
    clearable= True,
    multi= False,
)

# product dropdown
##Create options of products
options_products = [{'label': name + ' ', 'value': identifier} 
                    for name, identifier in zip(df['product_name_x'].unique(), 
                                                df['product_id'].unique())]
# options_products = [{'label': name + ' ', 'value': name} 
#                     for name in products['product_id']]
##Dropdown for products
product_dropdown = dcc.Dropdown(
    id='product_dropdown',
    options= options_products,
    value= '',
    style = {'width': '50%'},
    clearable= True,
    multi= True,
)

In [8]:
# Initialize the Dash app
app = dash.Dash(external_stylesheets=[dbc.themes.SKETCHY])

# Define the layout of the app
app.layout = dbc.Container([
    Navigation #Add the navigation bar
    ,
    dbc.Row(Carousel, 
            style={"margin-bottom": "20px", "maxHeight": "50%", "maxWidth": "100%"},
             )# Add carousel of images
    ,
# Add the table for top n trending products
    dbc.Row([
        dbc.Col([
            dbc.Row(Top_n_rated)
        ], width=6), 
        dbc.Col([
            dbc.Row(Top_n_trending)  
        ], width=6)
    ], style={'margin-right': '40px'}),
    html.Br(),

# Add personalized recommendations
    dbc.Row([
            html.H2('Personalized Recommendations', 
                        style = {'textAlign': 'center', 'margin-bottom': '7px'})]),
    dbc.Container([
        dbc.Row([dbc.Col(html.P('Please enter your user ID', 
                    style = {'textAlign': 'Left', 'margin-bottom': '7px'}), width= 5),
                dbc.Col(html.H5(''),
                        style = {'textAlign': 'left', 'margin-bottom': '7px'})]),
        dbc.Row([dbc.Col(user_id_dropdown, width= 5)
            ]),
        dbc.Row([
                dbc.Col(width= 2),
                dbc.Col([
                    html.Div(id='recommendations')
                ], width= 11, style= {'margin-top': '20px', 'center': 'center'}),
            ]),
    ], style = {
            'width': '100%',
            'maxHeight' : '850px',
            'overflowY' : 'auto',
            'margin': '20px auto 40px auto',
            'padding': '20px',
            'borderRadius': '8px',
            'border': '1px solid #ccc',
        }),
#Add recommendations based on product characteristics
    html.Br(),
    html.H2('Let us help you find the perfect product based on your preference!', 
            style = {'textAlign': 'center', 'margin-bottom': '5px'}),
    html.Br(),
    dbc.Container([
    dbc.Row([dbc.Col(html.P('1. Which color is your hair?'), width= 6),
             dbc.Col(hair_color_dropdown, width= 6)
              ],
             style = {'textAlign': 'left', 'margin-bottom': '15px'}),
    
    dbc.Row([dbc.Col(html.P('2. Which color is your eye?'), width= 6),
            dbc.Col(eye_color_dropdown, width= 6)],
            style = {'textAlign': 'left', 'margin-bottom': '15px'}),

    dbc.Row([dbc.Col(html.P('3. What type of skin do you have?'), width= 6),
            dbc.Col(skin_type_dropdown, width= 6)
             ], style = {'textAlign': 'left','margin-bottom': '15px'}),  

    dbc.Row([dbc.Col(html.P('4. Which category best describes your skin tone?'), width= 6),
            dbc.Col(skin_tone_dropdown, width= 6)
              ],
             style = {'textAlign': 'left', 'margin-bottom': '15px'}),
    dbc.Row([dbc.Col(html.P('5. Please select your price range!'), width= 6),
            dbc.Label("Price (USD)", style = {'textAlign': 'Center'}),
            price_slider,
              ],
             style = {'textAlign': 'left', 'margin-bottom': '5px'}),
    dbc.Row([dbc.Col(html.P('6. Do you love niche products? '), width= 6),
            dbc.Col(niche_dropdown, width= 6)
              ],
             style = {'textAlign': 'left', 'margin-bottom': '15px'}),

], style = {
            'width': '100%',
            'maxHeight' : '2050px',
            'overflowY' : 'auto',
            'margin': '20px auto 40px auto',
            'padding': '20px',
            'borderRadius': '8px',
            'border': '1px solid #ccc',
        }),
    html.Br(),
    dbc.Row([
        dbc.Col(
        dbc.Button("Click Here!", id = "submit-button", className="mr-1", 
                    n_clicks = 0, size = "lg",
                   style = {'align-items': 'center', 'justify-content': 'center', 
                            'marginTop': '20px', 'marginBottom': '20px', 'marginLeft': 'auto',
                             'marginRight': 'auto',"background-color": "pink", "border-color": "pink"}),
                    width={"size": 6},
                    style={'textAlign': 'center'}
    )],justify =  'center'),
    html.Br(),
    html.Div(
        [html.Div(dcc.Loading(
                    id = "loading",
                    type = "circle",
                    children = [
                        html.Div(id = 'recommendations_product'),],),
                style = {'display': 'flex', 'align-items': 'center', 'justify-content': 'center'}  
            )
        ],
        ),
    html.Br(),
    # Recommendations based on product on the cart
    dbc.Row([
            html.H2('Which product would you like to add to your cart?', 
                        style = {'textAlign': 'center', 'margin-bottom': '7px'})]),
    # dbc.Container([
    dbc.Row([dbc.Col(html.P('Please select your favourite products', 
                    style = {'textAlign': 'Left', 'margin-bottom': '7px'}), width= 5),
                ]),
    dbc.Row(dbc.Col(product_dropdown)),
    dbc.Row([
        dbc.Col(
        dbc.Button("Add to Bag", id = "submit-button-cart", className="mr-1", 
                    n_clicks = 0, size = "lg",
                   style = {'align-items': 'center', 'justify-content': 'center', 
                            'marginTop': '20px', 'marginBottom': '20px', 'marginLeft': 'auto',
                             'marginRight': 'auto',"background-color": "pink", "border-color": "pink"}),
                    width={"size": 3},
                    style={'textAlign': 'left'}
    ),
        dbc.Col(    
            html.Div(
                [html.Div(dcc.Loading(
                    id = "loading_cart",
                    type = "circle",
                    children = [
                        html.Div(id = 'similar_product_recommendations'),],),
                style = {'display': 'flex', 'align-items': 'left', 'justify-content': 'left'}  
            )
                ],
                ),)
            ],justify =  'left'),
    html.Br(),

], class_name= 'dbc',
    style = {'width': '100%', 'margin': '0 auto'})


# Callback to update the recommendations
@app.callback(
    Output('recommendations', 'children'),
    Input('user_id_dropdown', 'value'),
    prevent_initial_call = True
)

def update_recommendations(user_id):
    if user_id is None:
        return html.Div([
            html.P('Please enter your user ID to get personalized recommendations')
        ])
    else:
        recommendations_list = get_top_n_recommendations(KNNBasicrecommender, user_id, anti_set, 5, df)
        recommendations = get_top_n_recommendations_final(recommendations_list)
        return html.Div([
            recommendations
        ])

@app.callback(
    Output('recommendations_product', 'children'),
    Input('submit-button', 'n_clicks'),
    State('hair_color_dropdown', 'value'),
    State('eye_color_dropdown', 'value'),
    State('skin_type_dropdown', 'value'),
    State('skin_tone_dropdown', 'value'),
    State('input-price', 'value'),
    State('niche_dropdown', 'value'),
    State('user_id_dropdown', 'value'),
    prevent_initial_call = True
)

def update_recommendations_product(n_clicks, hair_color, eye_color, 
                                   skin_type, skin_tone, price, niche, user_id
                                     ):
    if user_id is None:
        return html.Div([
            html.P('Please enter your user ID to get personalized recommendations!')
        ])
    #Extract min and max price
    min_price, max_price = price
    # Store user input
    user_input = {
        'hair_color': hair_color,
        'eye_color': eye_color,
        'skin_type': skin_type,
        'skin_tone': skin_tone,
        'min_price': min_price,
        'max_price': max_price,
        'niche_product': niche
    }
    df_filtered = df.copy()
    #Filter the DataFrame based on user input
    if user_input['hair_color']:
        df_filtered = df_filtered[df_filtered['hair_color'].isin(hair_color)]
    if user_input['eye_color']:
        df_filtered = df_filtered[df_filtered['eye_color'].isin(eye_color)]
    if user_input['skin_type']:
        df_filtered = df_filtered[df_filtered['skin_type'].isin(skin_type)]
    if user_input['skin_tone']:
        df_filtered = df_filtered[df_filtered['skin_tone'].isin(skin_tone)]
    if user_input['min_price']:
        df_filtered = df_filtered[df_filtered['price_usd_x'] >= user_input['min_price']]
    if user_input['max_price']:
        df_filtered = df_filtered[df_filtered['price_usd_x'] <= user_input['max_price']]
    if user_input['niche_product']:
        df_filtered = df_filtered[df_filtered['niche_product'] == (niche - 1)]

    recommendations = user_profile_recommender(user_id, 5, df = df, df_filtered = df_filtered, products = products)
    return html.Div([
            recommendations
        ])

@app.callback(
    Output('similar_product_recommendations', 'children'),
    Input('submit-button-cart', 'n_clicks'),
    State('product_dropdown', 'value'),
    prevent_initial_call = True
)
def update_similar_product_recommendations(n_clicks, product):
    if product is None:
        return html.Div([
            html.P('Please select a product to add to your cart!')
        ])
    else:
        recommendations = get_similar_products(product, 5, df, products)
        return html.Div([
            recommendations
        ])
# Run the app and display result outline in the browser
app.run_server(port = 2024)